# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query ={"scores.Hygiene": 20}
fields = {"_id":1, "AddressLine1":1, "AddressLine2":1, "AddressLine3":1, "BusinessName":1, "BusinessType":1, "BusinessTypeID":1, 
          "PostCode":1, "RatingDate":1, "RatingValue":1, "geocode":1, "scores":1}
sort = [("scores.Hygiene", -1)]

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))
# Display the first document in the results using pprint
print()
pprint(establishments.find(query, fields).sort(sort)[0])

0



IndexError: no such item for Cursor instance

In [8]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query, fields).sort(sort)
df = pd.json_normalize(results)
# Display the number of rows in the DataFrame

print(df.shape)
# Display the first 10 rows of the DataFrame
df.head(10)

(0, 0)


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [10]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query ={"LocalAuthorityName": {"$regex": "London"},
        "RatingValue": {"$gte": 4}}
fields = {"_id":1, "AddressLine1":1, "AddressLine2":1, "AddressLine3":1, "BusinessName":1, "BusinessType":1, "BusinessTypeID":1, 
          "PostCode":1, "RatingDate":1, "RatingValue":1, "geocode":1, "scores":1}

sort = [("scores.Hygiene", -1)]
# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(establishments.find(query, fields).sort(sort)[0])

0


IndexError: no such item for Cursor instance

In [11]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query, fields).sort(sort)
df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print(df.shape)
# Display the first 10 rows of the DataFrame
df.head(10)

(0, 0)


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [12]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {"geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
        "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
        "RatingValue": 5}
fields = {"_id":1, "AddressLine1":1, "AddressLine2":1, "AddressLine3":1, "BusinessName":1, "BusinessType":1, "BusinessTypeID":1, 
          "PostCode":1, "RatingDate":1, "RatingValue":1, "geocode":1, "scores":1}
sort = [("scores.Hygiene", 1)] #, ("BusinessName", 1)]
limit = 5


# Print the results
print(establishments.count_documents(query))

0


In [13]:
# Convert result to Pandas DataFrame
results = establishments.find(query, fields).sort(sort).limit(limit)
df = pd.json_normalize(results)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [14]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {'scores.Hygiene': 0
                         }
              }

group_query = {'$group': {'_id': {
                                    'LocalAuthorityName': "$LocalAuthorityName"
                                },
                          'num_restaurants': { '$sum': 1 }, 
                         }
              }

sort_values = {'$sort': { 'num_restaurants': -1
                        }
              }

pipeline = [match_query, group_query, sort_values]

# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))

# Print the first 10 results
pprint(results[0:10])

[]


In [15]:
# Convert the result to a Pandas DataFrame
df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(df.shape)

# Display the first 10 rows of the DataFrame
df.head(10)

(0, 0)


""
